In [2]:
# conditional_pipleine.ipynb 

import kfp
from typing import NamedTuple
from collections import namedtuple
from kfp.components import create_component_from_func


@create_component_from_func
def sum_product_op(a: float, b: float) -> NamedTuple('TwoOutputs',
                                                     [('sum', float),
                                                      ('product', float)]):
    sum_result = a + b
    product_result = a * b
    
    # import 구문을 함수 안에 넣기.
    from collections import namedtuple
    
    two_output = namedtuple('TwoOutputs', ['sum', 'product'])
    return two_output(sum_result, product_result)


In [3]:
# conditional_pipleine.ipynb 
@create_component_from_func
def is_even_odd_op(num: float) -> NamedTuple('TwoOutputs',
                                             [('num', float),
                                              ('num_type', str)]):
    num_type = "even"
    if round(num) % 2 == 1:
        num_type = "odd"
        
    from collections import namedtuple

    two_outputs = namedtuple('TwoOutputs', ['num', 'num_type'])
    return two_outputs(num, num_type)

@create_component_from_func
def odd_op(num: float):
    print(f"{num} is odd")

@create_component_from_func
def even_op(num: float):
    print(f"{num} is even")


In [9]:
# conditional_pipleine.ipynb 
from kfp import dsl
client = kfp.Client() 

@dsl.pipeline(
    name='Conditional execution pipeline'
)
def is_odd_even_pipeline(a: float = 1, b: float = 1):
    sum_result = sum_product_op(a, b)
    
    print(sum_result.outputs['sum'])
    
    is_even_odd_result = is_even_odd_op(sum_result.outputs['sum'])
    
    num_type = is_even_odd_result.outputs['num_type']
    print(num_type)
    num = is_even_odd_result.outputs['num']
    print(num)
    
    with dsl.Condition(num_type == 'odd'):
        odd_op(num)
    with dsl.Condition(num_type == 'even'):
        even_op(num) 
        
arguments = {'a':'7','b': '8'}
client.create_run_from_pipeline_func(is_odd_even_pipeline, 
                                     arguments=arguments) 


{{pipelineparam:op=sum-product-op;name=sum}}
{{pipelineparam:op=is-even-odd-op;name=num_type}}
{{pipelineparam:op=is-even-odd-op;name=num}}


ApiException: (500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'trailer': 'Grpc-Trailer-Content-Type', 'date': 'Fri, 13 Aug 2021 05:47:35 GMT', 'x-envoy-upstream-service-time': '367', 'server': 'envoy', 'transfer-encoding': 'chunked'})
HTTP response body: {"error":"Failed to create a new run.: InternalServerError: Failed to store run conditional-execution-pipeline-f56tm to table: Error 1366: Incorrect string value: '\\xEA\\xB5\\xAC\\xEB\\xAC\\xB8...' for column 'WorkflowRuntimeManifest' at row 1","code":13,"message":"Failed to create a new run.: InternalServerError: Failed to store run conditional-execution-pipeline-f56tm to table: Error 1366: Incorrect string value: '\\xEA\\xB5\\xAC\\xEB\\xAC\\xB8...' for column 'WorkflowRuntimeManifest' at row 1","details":[{"@type":"type.googleapis.com/api.Error","error_message":"Internal Server Error","error_details":"Failed to create a new run.: InternalServerError: Failed to store run 